In [0]:
%%sh
# Download the CMS Open Payments ZIP file
curl -L https://download.cms.gov/openpayments/PGYR2023_P01302025_01212025.zip -o /tmp/openpayments_2023.zip

# Unzip the file to a directory, force overwriting existing files without prompting
unzip -o /tmp/openpayments_2023.zip -d /tmp/openpayments_2023

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  752M  100  752M    0     0   127M      0  0:00:05  0:00:05 --:--:--  111M


Archive:  /tmp/openpayments_2023.zip
  inflating: /tmp/openpayments_2023/OP_PGYR2023_README_P01302025.txt  
  inflating: /tmp/openpayments_2023/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv  


In [0]:
# Importing required libraries
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc, count


In [0]:
import os

# List the extracted files
os.listdir("/tmp/openpayments_2023")

Out[53]: ['OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv',
 'OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv',
 'OP_PGYR2023_README_P01302025.txt']

In [0]:
# Reading the CSV file into a Spark DataFrame
df = spark.read.csv("file:/tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv", header=True, inferSchema=True)

# Saving it as a Delta table
df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("greeshma_week7_General_openPays_2023")

In [0]:
# Reading the CSV file into a Spark DataFrame
df = spark.read.csv("dbfs:/FileStore/shared_uploads/greeshma.poli@slu.edu/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv")

# Saving it as a Delta table (use underscores instead of spaces)
df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("Cp_covered_recipients")

In [0]:
# 1. Nature of Payments with reimbursement amounts > $1,000 ordered by count
nature_over_1000 = (df_open_payments
    .filter(col("total_amount_of_payment_usdollars") > 1000)
    .groupBy("nature_of_payment_or_transfer_of_value")
    .agg(count("nature_of_payment_or_transfer_of_value").alias("count"))
    .orderBy(desc("count"))
)
nature_over_1000.show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|nature_of_payment_or_transfer_of_value                                                                                                            |count |
+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|164092|
|Consulting Fee                                                                                                                                    |105228|
|Travel and Lodging                                                                                                                                |24738 |
|Honoraria                                                      

In [0]:
# 2. Top 10 Nature of Payments by count
top_10_nature_by_count = (df_open_payments
    .groupBy("nature_of_payment_or_transfer_of_value")
    .agg(count("nature_of_payment_or_transfer_of_value").alias("count"))
    .orderBy(desc("count"))
)
top_10_nature_by_count.show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|nature_of_payment_or_transfer_of_value                                                                                                            |count   |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Food and Beverage                                                                                                                                 |13378464|
|Travel and Lodging                                                                                                                                |545086  |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|236628  |
|Consulting Fee                                     

In [0]:
# 3. Top 10 Nature of Payments by total amount
top_10_nature_by_amount = (df_open_payments
    .groupBy("nature_of_payment_or_transfer_of_value")
    .agg(sum("total_amount_of_payment_usdollars").alias("total_amount"))
    .orderBy(desc("total_amount"))
)
top_10_nature_by_amount.show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|nature_of_payment_or_transfer_of_value                                                                                                            |total_amount        |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Royalty or License                                                                                                                                |1.19217456302E9     |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|5.946326876500002E8 |
|Consulting Fee                                                                                                                                    |5.

In [0]:
# 4. Top 10 physician specialties by total amount
top_10_specialties_by_amount = (df_open_payments
    .groupBy("Covered_Recipient_Specialty_1")
    .agg(sum(col("Total_Amount_of_Payment_USDollars").cast("double")).alias("total_amount"))
    .orderBy(desc("total_amount"))
)
top_10_specialties_by_amount.show(10, truncate=False)


+------------------------------------------------------------------------------------------------+--------------------+
|Covered_Recipient_Specialty_1                                                                   |total_amount        |
+------------------------------------------------------------------------------------------------+--------------------+
|null                                                                                            |7.936674692300001E8 |
|Allopathic & Osteopathic Physicians|Orthopaedic Surgery                                         |4.0345021308999825E8|
|Allopathic & Osteopathic Physicians|Internal Medicine                                           |1.3136300307000063E8|
|Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Neurology                            |8.979213626000014E7 |
|Allopathic & Osteopathic Physicians|Neurological Surgery                                        |8.608847857000013E7 |
|Allopathic & Osteopathic Physicians|Der

In [0]:
df_open_payments.printSchema()


root
 |-- Change_Type: string (nullable = true)
 |-- Covered_Recipient_Type: string (nullable = true)
 |-- Teaching_Hospital_CCN: string (nullable = true)
 |-- Teaching_Hospital_ID: string (nullable = true)
 |-- Teaching_Hospital_Name: string (nullable = true)
 |-- Covered_Recipient_Profile_ID: string (nullable = true)
 |-- Covered_Recipient_NPI: string (nullable = true)
 |-- Covered_Recipient_First_Name: string (nullable = true)
 |-- Covered_Recipient_Middle_Name: string (nullable = true)
 |-- Covered_Recipient_Last_Name: string (nullable = true)
 |-- Covered_Recipient_Name_Suffix: string (nullable = true)
 |-- Recipient_Primary_Business_Street_Address_Line1: string (nullable = true)
 |-- Recipient_Primary_Business_Street_Address_Line2: string (nullable = true)
 |-- Recipient_City: string (nullable = true)
 |-- Recipient_State: string (nullable = true)
 |-- Recipient_Zip_Code: string (nullable = true)
 |-- Recipient_Country: string (nullable = true)
 |-- Recipient_Province: string (nu

In [0]:
# 5.  The top ten physicians by total amount
top_10_physicians_by_amount = (
    df_open_payments
    .filter(col("Covered_Recipient_First_Name").isNotNull() & col("Covered_Recipient_Last_Name").isNotNull())  # Only non-null names
    .groupBy("Covered_Recipient_First_Name", "Covered_Recipient_Last_Name")
    .agg(sum(col("Total_Amount_of_Payment_USDollars").cast("double")).alias("total_amount"))
    .orderBy(desc("total_amount"))
)

top_10_physicians_by_amount.show(10, truncate=False)


+----------------------------+---------------------------+-----------------+
|Covered_Recipient_First_Name|Covered_Recipient_Last_Name|total_amount     |
+----------------------------+---------------------------+-----------------+
|STEPHEN                     |BURKHART                   |3.392202493E7    |
|WILLIAM                     |BINDER                     |2.943437497E7    |
|KEVIN                       |FOLEY                      |1.73059378E7     |
|IVAN                        |OSORIO                     |1.606551551E7    |
|GEORGE                      |MAXWELL                    |1.160032024E7    |
|ROBERT                      |BOOTH                      |8459167.19       |
|NEAL                        |ELATTRACHE                 |7810628.2        |
|AARON                       |ROSENBERG                  |6883627.290000001|
|ROGER                       |JACKSON                    |6615711.26       |
|PETER                       |BONUTTI                    |6385096.170000001|